# Canonical ensemble Monte Carlo of bulk SPC/E water


In this example, we reproduce the average energy reported in https://doi.org/10.1063/1.476834

In [5]:
import unittest
import feasst as fst

class TestEwald1SPCENVT(unittest.TestCase):
    """Test a canonical ensemble Lennard Jones Monte Carlo simulation"""
    def test_srsw(self, num_particles=512, box_length=24.8586887, trials_per=1e5, temperature=298):
        """Compare with the reported average energy from https://doi.org/10.1063/1.476834.

        num_particles -- number of water molecules
        box_length -- box length in angstroms
        trials_per -- steps between each Anaylze/Modify
        temperature -- in Kelvin
        """

        monte_carlo = fst.MonteCarlo()
        # monte_carlo.set(fst.MakeRandomMT19937(fst.args({"seed": "1234"})))
        # Original manuscript used rcut = L/2, but the default rcut in spce used here is 10
        monte_carlo.add(fst.MakeConfiguration(fst.args({"cubic_box_length": str(box_length),
            "physical_constants": "CODATA2010",
            "particle_type0": fst.install_dir() + "/forcefield/spce.fstprt"})))
        monte_carlo.add(fst.MakePotential(fst.MakeEwald(fst.args({"alpha": str(5.6/box_length),
            "kmax_squared": "38"}))))
        monte_carlo.add(fst.MakePotential(fst.MakeModelTwoBodyFactory(fst.MakeLennardJones(), fst.MakeChargeScreened()),
                                          fst.args({"table_size": "1e6"})))
        monte_carlo.add(fst.MakePotential(fst.MakeChargeScreenedIntra(), fst.MakeVisitModelBond()))
        monte_carlo.add(fst.MakePotential(fst.MakeChargeSelf()))
        monte_carlo.add(fst.MakePotential(fst.MakeLongRangeCorrections()))
        R = monte_carlo.configuration().physical_constants().ideal_gas_constant()
        monte_carlo.set(fst.MakeThermoParams(fst.args({
            "beta": str(1./(R*temperature/1e3)),
            "chemical_potential": "1."})))
        monte_carlo.set(fst.MakeMetropolis())
        monte_carlo.add(fst.MakeTrialTranslate(fst.args({"weight": "1.", "tunable_param": "0.275"})))
        monte_carlo.add(fst.MakeTrialRotate(fst.args({"weight": "1.", "tunable_param": "0.5"})))
        monte_carlo.add(fst.MakeTrialAdd(fst.args({"particle_type": "0"})))
        monte_carlo.run(fst.MakeRun(fst.args({"until_num_particles": str(num_particles)})))
        monte_carlo.run(fst.MakeRemoveTrial(fst.args({"name": "TrialAdd"})))
        monte_carlo.add(fst.MakeLogAndMovie(fst.args({
            "trials_per" : str(trials_per),
            "file_name": "spce",
            "clear_file": "true"})))
        monte_carlo.add(fst.MakeCheckEnergyAndTune(fst.args({"trials_per" : str(trials_per),
                                                             "tolerance": str(1e-6)})))

        # equilibrate
        monte_carlo.attempt(int(1e6))

        # compute average energy using a stepper/analysis and output into file
        energy = fst.MakeEnergy(fst.args({
            "trials_per_update": "1",
            "trials_per_write": str(trials_per),
            "file_name": "spce_nvt_energy.txt"}))
        monte_carlo.add(energy)

        # compute average using this script
        energy_alt = fst.Accumulator()

        # production
        for _ in range(int(1e6)):
            monte_carlo.attempt()
            energy_alt.accumulate(monte_carlo.criteria().current_energy())

        # test that the two methods to compute average energy are the same
        self.assertAlmostEqual(energy.energy().average(), energy_alt.average(), delta=1e-6)

        # test the average against the NIST SRSW
        num = monte_carlo.configuration().num_particles()
        stdev = (energy.energy().block_stdev()**2 + (0.02*num)**2)**(1./2.)
        print(energy.energy().average(), energy_alt.average(), stdev)
        self.assertAlmostEqual(-46.82*num, energy.energy().average(), delta=20*stdev)

If the test passes, the energy is within the tolerance of the SRSW value and the two ensemble average methods agreed.

In [ ]:
%time
unittest.main(argv=[''], verbosity=2, exit=False)

test_srsw (__main__.TestEwald1SPCENVT)
Compare with the reported average energy from https://doi.org/10.1063/1.476834. ... 

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!